In [297]:
__author__ = "me"
__date__ = "2015_10_13"

%pylab inline
import pandas as pd
import numpy as np
import random

import pylab as plt
import os

from fuzzywuzzy import process

import requests
s = requests.get("https://raw.githubusercontent.com/Casyfill/CUSP_templates/master/Py/fbMatplotlibrc.json").json()
plt.rcParams.update(s)

numpy.random.seed(2015)

Populating the interactive namespace from numpy and matplotlib


`%matplotlib` prevents importing * from pylab and numpy


## PropID

In [267]:
PARQA = os.getenv('PARQA')

In [268]:
prop = pd.read_excel(PARQA + 'data/Input/Parks_Data/CUSP_Adjusted_Spatial_Data.xlsx')[['ParkID','Name']]

In [269]:
prop = prop.dropna()

In [270]:
def trySplit(x, spl='-'):
    if spl in x:
        return x.split(spl)[0].strip()
    else:
        return x

In [271]:
prop.ParkID = prop.ParkID.apply(trySplit)
prop.Name = prop.Name.apply(lambda x: trySplit(x, spl='Zone'))
prop['boro'] = prop.ParkID.str[0]

In [272]:
prop = prop.drop_duplicates()
prop.Name = prop.Name.str.lower()

In [273]:
prop.ParkID[(prop.ParkID.str.len() > 4)] = prop.ParkID[(prop.ParkID.str.len() > 4)].apply(lambda x: x[:4])


In [274]:
for name, g in prop.groupby('ParkID'):
    if len(g)>2:
        print g
        print

     ParkID                 Name boro
227    B018        dinapoli plgd    B
305    B018        canarsie park    B
2165   B018  canarsie playground    B

    ParkID                Name boro
17    B028          dyker plgd    B
222   B028    bay 8 playground    B
309   B028  dyker beach park -    B

    ParkID               Name boro
6     B057        ps 278 plgd    B
24    B057     dr john's plgd    B
27    B057  lenape playground    B
240   B057    seba playground    B
323   B057        marine park    B

    ParkID                                   Name boro
29    B073                       garfield tot lot    B
30    B073                          third st plgd    B
31    B073                        lincoln rd plgd    B
32    B073                     harmony playground    B
33    B073                       imagination plgd    B
167   B073                  vanderbilt playground    B
346   B073                          main entrance    B
347   B073                     prospect park west  

## Matching name to property

In [275]:
prop['NAME'] = prop.Name

In [278]:
prop.NAME[(prop.ParkID.str.contains('B073'))] = 'prospect park'
prop.NAME[(prop.ParkID.str.contains('M010'))] = 'central park'
prop.NAME[(prop.ParkID.str.contains('Q004'))] = 'astoria park'
prop.NAME[(prop.ParkID.str.contains('X010')) & (prop.NAME.str.contains('crotona'))] = 'crotona parks'
prop.NAME[(prop.ParkID.isin(['Q164','Q163','Q162','Q050']))] = 'rockaway beach boardwalk'
prop.NAME[(prop.ParkID.isin(['Q300','Q300A']))] = 'kissena corridor park'
prop.NAME[(prop.ParkID.isin(['M014']))] = 'jackie robinson park'
prop.NAME[(prop.ParkID.isin(['M028']))] = 'fort washington park'
prop.NAME[(prop.ParkID.isin(['M037']))] = 'highbridge park'
prop.NAME[(prop.ParkID.isin(['M098']))] = 'washington square park'
prop.NAME[(prop.ParkID.isin(['M105']))] = 'sara d. roosevelt park'
prop.NAME[(prop.ParkID.isin(['M107']))] = "hell's kitchen park"
prop.NAME[(prop.ParkID.isin(['M283']))] = "battery park city"
prop.NAME[(prop.ParkID.isin(['Q001']))] = "alley pond park"
prop.NAME[(prop.ParkID.isin(['Q005']))] = "baisley pond park"
prop.NAME[(prop.ParkID.isin(['Q009']))] = "macneil park"
prop.NAME[(prop.ParkID.isin(['Q012']))] = "crocheron park"
prop.NAME[(prop.ParkID.isin(['Q020']))] = "highland park"
prop.NAME[(prop.ParkID.isin(['Q021']))] = "cunningham park"
prop.NAME[(prop.ParkID.isin(['Q024']))] = "kissena park"
prop.NAME[(prop.ParkID.isin(['Q102']))] = "juniper valley park"


# kissena corridor park
# kissena park golf course

In [280]:
prop.head()

,ParkID,Name,boro,NAME
0,M058,marcus garvey memorial park,M,marcus garvey memorial park
2,M058,mt. morris east,M,mt. morris east
3,M047,thomas jefferson park,M,thomas jefferson park
4,M273,othmar ammann playground,M,othmar ammann playground
5,M081,catbird playground,M,catbird playground


In [283]:
DPRprop = prop[['NAME','Name','ParkID','boro']].groupby('NAME').agg({'Name': lambda x:list(set(x.tolist())), 'ParkID':lambda x: list(set(x.tolist())),'boro':'first'}).reset_index()

In [284]:
DPRprop.head()

,NAME,boro,ParkID,Name
0,"""uncle"" vito e. maranzano glendale playground",Q,[Q290],"[""uncle"" vito e. maranzano glendale playground]"
1,100% playground,B,[B342],[100% playground]
2,101st st. soccer field,M,[M071],[101st st. soccer field]
3,104th st. softball field,M,[M071],[104th st. softball field]
4,108th st. softball field,M,[M071],[108th st. softball field]


In [285]:
names = prop.NAME.tolist()

In [286]:
prop.to_excel(PARQA + 'data/Input/Parks_Data/prop.xlsx')

## 311 Property Names

In [287]:
onto = pd.read_csv(PARQA + 'parqa/311/ONTOLOGY/onto_data/Ontology_matched.csv', index_col=0)
onto['boro'] = onto.parkDistrict.apply(trySplit)
# onto.head(3)

In [288]:
def getBest(x):
    return process.extractOne(x, names)

In [289]:
onto['bestMatch'] = onto.NAME.apply(getBest)

In [290]:
# for x in onto.NAME.unique().tolist(): print x

In [291]:
onto['conf'] = onto.bestMatch.str.get(1)
onto.bestMatch = onto.bestMatch.str.get(0)
onto.conf[onto.bestMatch=='playground'] = 50 ## this not gonna help us

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  app.launch_new_instance()


In [292]:
onto.conf.describe()

count    1793.000000
mean       91.554936
std        23.398561
min         0.000000
25%        95.000000
50%       100.000000
75%       100.000000
max       100.000000
Name: conf, dtype: float64

#### For now: Work only with those with confidence score >90

In [293]:
ontoConf = onto[onto.conf>90]

## Group DPR Property by park name (for randomization)

In [298]:
DPR = ontoConf.merge(DPRprop, on=['NAME','boro'], how='left')[['cleanName','bestMatch','ParkID']]

In [295]:
# DPR.head()
DPR[DPR.ParkID.str.len() >1 ]

,cleanName,NAME,Type,valid,parkDistrict,lat,lon,boro,bestMatch,conf,ParkID,Name
42,hells kitchen park,hell's kitchen park,other,av,M-04,-73.992583,40.762968,M,hell's kitchen park,100,"[M277, M107]","[hell's kitchen park, hell gate, wards meadow,..."
75,riverside park - claremont playground,claremont,pgs,av,M-14,-73.962100,40.815000,M,claremont,100,"[M071, X008]",[claremont]
76,hillside park - jhs 172,hillside park,other,v,B-02,-73.994280,40.701401,B,hillside park,100,"[Q352, B223]",[hillside park]
118,east river playground - ps 146,east river playground,pgs,av,M-11,-73.938200,40.789400,M,east river playground,100,"[M144, M258]",[east river playground]
126,east river park,east river esplanade,other,v,M-06,-73.970502,40.744978,M,east river esplanade,100,"[M289, M108]",[east river esplanade]
127,east river park,east river esplanade,other,v,M-11,-73.936332,40.789640,M,east river esplanade,100,"[M289, M108]",[east river esplanade]
143,jackie robinson park - playground one fifty tw...,jackie robinson park,pgs,av,M-10,-73.941200,40.825900,M,jackie robinson park,100,"[M014, B294]","[jackie robinson park, plgd one fifty two clii..."
147,rockaway beach boardwalk,rockaway beach boardwalk,park_direct,av,Q-14,-73.814644,40.581159,Q,rockaway beach boardwalk,100,"[Q050, Q164, Q162, Q163]","[beach 30th street playground, rockaway beach ..."
176,highbridge park,highbridge park,park_direct,av,M-12A,-73.930465,40.846501,M,highbridge park,100,"[M037, X120]","[ft george plgd, dyckman sitting area, cpf plg..."
178,highbridge park,highbridge park,park_direct,av,M-12A,-73.930465,40.846501,M,highbridge park,100,"[M037, X120]","[ft george plgd, dyckman sitting area, cpf plg..."


#### now for those where there are multiple ParkID we need to randomly pick one or another for each call

## Matching calls

In [301]:
calls = pd.read_csv(PARQA + 'data/311/311DPR.csv', encoding='utf8', na_values='Unspecified')

myCalls = calls[['Park Facility Name','Descriptor','Created Date','Closed Date','Longitude','Latitude','Location Type', 'Complaint Type']]
myCalls['Park Facility Name'] = myCalls['Park Facility Name'].str.lower()
myCalls['Park Facility Name'].head()

notNamedCalls = myCalls[pd.isnull(myCalls['Park Facility Name'])]
namedCalls = myCalls[pd.notnull(myCalls['Park Facility Name'])]

/Users/casy/anaconda/lib/python2.7/site-packages/ipykernel/__main__.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy


In [302]:
ncMatched = namedCalls.merge(DPR, how='left',left_on='Park Facility Name', right_on='cleanName')
ncMatched.head(1)

,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type,Complaint Type,cleanName,bestMatch,ParkID
0,geo soilan park - battery park city,Graffiti or Vandalism,12/31/2010 04:31:52 PM,12/31/2010 05:36:58 PM,NaN,NaN,Park,Maintenance or Facility,geo soilan park - battery park city,battery park city,[M283]


In [307]:
DPR[DPR.cleanName =='highland park']

,cleanName,bestMatch,ParkID
3,highland park,highland park,NaN
4,highland park,highland park,[Q020]
5,highland park,highland park,NaN
6,highland park,highland park,[Q020]


In [306]:
# ncMatched['sParkID'] = nc.Matched
ncMatched[pd.isnull(ncMatched['ParkID'])].head()


,Park Facility Name,Descriptor,Created Date,Closed Date,Longitude,Latitude,Location Type,Complaint Type,cleanName,bestMatch,ParkID
3,highland park,Snow or Ice,12/31/2010 02:57:34 PM,01/03/2011 11:31:26 AM,NaN,NaN,Park,Maintenance or Facility,highland park,highland park,NaN
5,highland park,Snow or Ice,12/31/2010 02:57:34 PM,01/03/2011 11:31:26 AM,NaN,NaN,Park,Maintenance or Facility,highland park,highland park,NaN
13,col david marcus playground,Snow or Ice,12/31/2010 09:12:19 AM,01/03/2011 12:16:41 PM,NaN,NaN,Park,Maintenance or Facility,NaN,NaN,NaN
17,mccarren park - vincent v abate playground,Snow or Ice,12/30/2010 10:12:25 PM,01/03/2011 02:28:11 PM,NaN,NaN,Park,Maintenance or Facility,NaN,NaN,NaN
20,forest park - giovannelli playground,Dog Off Leash,12/30/2010 09:31:36 PM,12/30/2010 09:59:44 PM,NaN,NaN,Park,Animal in a Park,NaN,NaN,NaN


## HERE I NEED TO SPATIALLY JOIN GEOLOCATED ONES